In [1]:
import pandas as pd
from mwviews.api import PageviewsClient
import numpy as np
import chardet

def get_direct_link(shared_link):
    return 'https://drive.google.com/uc?export=download&id=' + shared_link.split('/')[-2]

link = "https://drive.google.com/file/d/1ndN5kFHdH1yhblgRUWdaFYicwt3WtcTB/view?usp=drive_link"

direct_link = get_direct_link(link)
teams_df = pd.read_csv(direct_link)

end_date = '20230603'

In [2]:
def get_team_data(team):
    results = pd.DataFrame(columns=['Date', 'wiki', 'Team'])
    for team in team:
        try:
            data = client.article_views('en.wikipedia', team,
                                        granularity='daily',
                                        start='20150104',
                                        end=end_date)
        except:
            print("Kinda error")
            continue

        data_df = pd.DataFrame.from_dict(data, orient='index')
        column = data_df.columns[0]
        data_df = data_df.reset_index()
        data_df.rename(columns={'index' : 'Date',
                                 column : 'wiki'}, inplace=True)
        data_df['Team'] = team

        date_mask = data_df['Date'].isin(results['Date'])
        team_mask = data_df['Team'].isin(results['Team'])
        dup_mask = date_mask & team_mask
        new_data = data_df[~dup_mask]
        if new_data.shape[0] == 0:
            continue

        new_data.fillna(0, inplace=True)
        new_data['year'] = new_data['Date'].dt.year
        new_data['month'] = new_data['Date'].dt.month
        new_data['day'] = new_data['Date'].dt.day
        results = pd.concat([results,new_data])

    return results

In [3]:
def combine_duplicate_teams(df):
    grouped_teams = df.groupby('wiki_title')['original_names'].apply(np.unique).reset_index()

    grouped_teams['original_names'] = grouped_teams['original_names'].apply(tuple)

    merged = df.merge(grouped_teams, on='wiki_title', how='left')

    merged.drop(columns='original_names_x', inplace=True)
    merged.rename(columns={'original_names_y': 'original_names'}, inplace=True)

    merged.drop_duplicates(inplace=True)

    return merged


In [4]:
teams = list(teams_df['wiki_title'].dropna().unique())

client = PageviewsClient(user_agent="<jg19184@bristol.ac.uk> Buzz Factor Research")

results = get_team_data(teams)

merged = teams_df[['original_names', 'wiki_title']].merge\
  (results, left_on='wiki_title', right_on='Team',  how='left')
merged.drop(columns=['Team'], inplace=True)

print('There are this many null values:',merged.wiki.isnull().values.sum())
print('These are the null values:', merged.loc[merged.wiki.isnull()==True].wiki_title.values)

merged = merged[merged.wiki.notnull()]
nulls = merged.groupby('Date')['wiki'].sum() == 0
null_dates = nulls[nulls == True].index.tolist()
print('Dates with no views:', nulls.loc[nulls == True].index)
merged = merged[~merged.Date.isin(null_dates)]

merged = combine_duplicate_teams(merged)

merged.sort_values(['wiki_title', 'Date'], inplace=True)
merged.to_csv('football_wikipedia.csv', encoding='utf8', index=False)

ERROR while fetching and parsing ['https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia/all-access/all-agents/AC_Carpi/daily/2015010400/2023060300']
Kinda error


Traceback (most recent call last):
  File "C:\Users\Archie Stevens\Documents\Anaconda\lib\site-packages\mwviews\api\pageviews.py", line 146, in article_views
    raise Exception(
Exception: The pageview API returned nothing useful at: ['https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia/all-access/all-agents/AC_Carpi/daily/2015010400/2023060300']


ERROR while fetching and parsing ['https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia/all-access/all-agents/US_Salernitana_1919/daily/2015010400/2023060300']
Kinda error


Traceback (most recent call last):
  File "C:\Users\Archie Stevens\Documents\Anaconda\lib\site-packages\mwviews\api\pageviews.py", line 146, in article_views
    raise Exception(
Exception: The pageview API returned nothing useful at: ['https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia/all-access/all-agents/US_Salernitana_1919/daily/2015010400/2023060300']


There are this many null values: 2
These are the null values: ['AC Carpi' 'US Salernitana 1919']
Dates with no views: DatetimeIndex(['2015-01-04', '2015-01-05', '2015-01-06', '2015-01-07',
               '2015-01-08', '2015-01-09', '2015-01-10', '2015-01-11',
               '2015-01-12', '2015-01-13',
               ...
               '2015-06-21', '2015-06-22', '2015-06-23', '2015-06-24',
               '2015-06-25', '2015-06-26', '2015-06-27', '2015-06-28',
               '2015-06-29', '2015-06-30'],
              dtype='datetime64[ns]', name='Date', length=178, freq=None)
